In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 
import os
plt.rcParams['font.family'] = 'AppleGothic' 
plt.rcParams['axes.unicode_minus'] = False
import koreanize_matplotlib

In [3]:
hackle_df0 = pd.read_parquet("/home/sprintda05/final/hackle/2023-04-30.parquet")

In [3]:
hackle_df0

,Asia/Seoul,event_key,session_id,user_id,value,user_properties,hackle_properties,event_properties,id,device_id
0,2023-05-01T00:00:00.024Z,button,jgGFnoyc6GWuGIEgjp3nUoKRQbF3,None,0.0,[],"{'language': 'ko-KR', 'isapp': 'true', 'osvers...","{'description': '다음', 'location': 'signotherin...",96d1336f-744b-4deb-a6a4-207494419dbf,cfc34eab-6930-488b-8109-70d37cec7dae
1,2023-05-01T00:00:00.025Z,button,n3PI8GZnRFOKpJslIRgckMeRiln1,None,0.0,[],"{'language': 'ko-CA', 'isapp': 'true', 'osvers...","{'description': 'vote 선택', 'location': 'homevo...",2825d1b1-8325-42e2-88eb-8cae61d3a26d,0dde2ecb-ff03-440f-98f3-94f20978b796
2,2023-05-01T00:00:00.029Z,button,L31Dl8vRtdWPJ8XC0gGs1mPwzJH2,None,0.0,[],"{'language': 'ko', 'isapp': 'true', 'osversion...","{'description': '친구선택', 'location': 'homeenque...",335b0a36-9ecf-41a9-b7d2-e27125e521ef,b50774ed-24ce-49e1-9383-945ff3aea9ff
3,2023-05-01T00:00:00.036Z,button,QjToElcYNkVxszJDHBhqtITlzDJ3,None,0.0,[],"{'language': 'ko', 'isapp': 'true', 'osversion...","{'description': '이름 셔플!', 'location': 'homeenq...",b730bb6a-589a-4074-89a1-87f11a03f9d6,0999c6b6-c579-4c0f-916c-9a113010f84a
4,2023-05-01T00:00:00.063Z,button,VRpXf303RGU730fIO4EjWstzAJi1,None,0.0,[],"{'language': 'ko-KR', 'isapp': 'true', 'osvers...","{'description': '앱바 뒤로가기', 'location': 'homefr...",c2a5fd3f-92d5-4864-8e0e-63e563898499,45985493-ed35-4cc5-935f-1bcf922e213c
...,...,...,...,...,...,...,...,...,...,...
8588417,2023-05-06T23:59:59.168Z,$session_end,96348488-4d6a-49a8-8995-452718157fd7,None,0.0,[],"{'language': 'ko', 'isapp': 'true', 'osversion...","{'description': None, 'location': None, 'sessi...",8f2fe201-6e6a-49c8-9988-b6f7ac540372,96348488-4d6a-49a8-8995-452718157fd7
8588418,2023-05-06T23:59:59.496Z,$session_end,9e9c7a7a-5efc-470b-8ecf-e3f6a102bda6,None,0.0,[],"{'language': 'ko', 'isapp': 'true', 'osversion...","{'description': None, 'location': None, 'sessi...",c0eaf066-9638-41f2-88cd-2dddca84ac4b,9e9c7a7a-5efc-470b-8ecf-e3f6a102bda6
8588419,2023-05-06T23:59:59.543Z,$session_end,60DD632C-010F-4689-A997-1E17E7368C9F,None,0.0,[],"{'language': 'ko-KR', 'isapp': 'true', 'osvers...","{'description': None, 'location': None, 'sessi...",76ae7486-ca23-4911-9f9b-ceed6b4b0f23,60dd632c-010f-4689-a997-1e17e7368c9f
8588420,2023-05-06T23:59:59.602Z,$session_start,lfu6e5HTwdQvbkSmOHppzJA585N2,None,0.0,[],"{'language': 'ko-KR', 'isapp': 'true', 'osvers...","{'description': None, 'location': None, 'sessi...",f460a877-eba1-45bd-9fbf-2604efce6d82,d9246f1a-f9cf-440f-ba36-07ed71836c4a


In [4]:
#  컬럼명 정리
hackle_df0 = hackle_df0.rename(columns={"Asia/Seoul": "datetime"})

#  $session_start만 필터링
session_start_df = hackle_df0[hackle_df0["event_key"] == "$session_start"].copy()

# datetime을 UTC로 간주 후 → KST로 변환
session_start_df["datetime"] = pd.to_datetime(session_start_df["datetime"], utc=True)
session_start_df["datetime_kst"] = session_start_df["datetime"].dt.tz_convert("Asia/Seoul")

# 날짜만 추출 (KST 기준)
session_start_df["date"] = session_start_df["datetime_kst"].dt.date

#  날짜별 유니크한 device_id 개수 집계
daily_device_counts = (
    session_start_df.groupby("date")["device_id"]
    .nunique()
    .reset_index()
    .rename(columns={"device_id": "unique_device_count"})
)

daily_device_counts.to_parquet("4.30_session_start_device_counts.parquet", index=False)


In [6]:
daily_device_counts

,date,unique_device_count
0,2023-05-01,18379
1,2023-05-02,22335
2,2023-05-03,22520
3,2023-05-04,22793
4,2023-05-05,27805
5,2023-05-06,67787


In [5]:
daily_device_counts

,date,unique_device_count
0,2023-05-01,11539
1,2023-05-02,19680
2,2023-05-03,23066
3,2023-05-04,22800
4,2023-05-05,23659
5,2023-05-06,34429
6,2023-05-07,59610


In [2]:
hackle_df1 = pd.read_parquet("/home/sprintda05/final/hackle/2023-05-06.parquet")

In [3]:
#  컬럼명 정리
hackle_df1 = hackle_df1.rename(columns={"Asia/Seoul": "datetime"})

#  $session_start만 필터링
session_start_df = hackle_df1[hackle_df1["event_key"] == "$session_start"].copy()

# datetime을 UTC로 간주 후 → KST로 변환
session_start_df["datetime"] = pd.to_datetime(session_start_df["datetime"], utc=True)
session_start_df["datetime_kst"] = session_start_df["datetime"].dt.tz_convert("Asia/Seoul")

# 날짜만 추출 (KST 기준)
session_start_df["date"] = session_start_df["datetime_kst"].dt.date

#  날짜별 유니크한 device_id 개수 집계
daily_device_counts = (
    session_start_df.groupby("date")["device_id"]
    .nunique()
    .reset_index()
    .rename(columns={"device_id": "unique_device_count"})
)

daily_device_counts.to_parquet("5.06_session_start_device_counts.parquet", index=False)

In [3]:
hackle_df2 = pd.read_parquet("/home/sprintda05/final/hackle/2023-05-12.parquet")

In [4]:
#  컬럼명 정리
hackle_df2 = hackle_df2.rename(columns={"Asia/Seoul": "datetime"})

#  $session_start만 필터링
session_start_df = hackle_df2[hackle_df2["event_key"] == "$session_start"].copy()

# datetime을 UTC로 간주 후 → KST로 변환
session_start_df["datetime"] = pd.to_datetime(session_start_df["datetime"], utc=True)
session_start_df["datetime_kst"] = session_start_df["datetime"].dt.tz_convert("Asia/Seoul")

# 날짜만 추출 (KST 기준)
session_start_df["date"] = session_start_df["datetime_kst"].dt.date

#  날짜별 유니크한 device_id 개수 집계
daily_device_counts = (
    session_start_df.groupby("date")["device_id"]
    .nunique()
    .reset_index()
    .rename(columns={"device_id": "unique_device_count"})
)

daily_device_counts.to_parquet("5.12_session_start_device_counts.parquet", index=False)

In [3]:
hackle_df3 = pd.read_parquet("/home/sprintda05/final/hackle/2023-05-18.parquet")

In [4]:
#  컬럼명 정리
hackle_df3 = hackle_df3.rename(columns={"Asia/Seoul": "datetime"})

#  $session_start만 필터링
session_start_df = hackle_df3[hackle_df3["event_key"] == "$session_start"].copy()

# datetime을 UTC로 간주 후 → KST로 변환
session_start_df["datetime"] = pd.to_datetime(session_start_df["datetime"], utc=True)
session_start_df["datetime_kst"] = session_start_df["datetime"].dt.tz_convert("Asia/Seoul")

# 날짜만 추출 (KST 기준)
session_start_df["date"] = session_start_df["datetime_kst"].dt.date

#  날짜별 유니크한 device_id 개수 집계
daily_device_counts = (
    session_start_df.groupby("date")["device_id"]
    .nunique()
    .reset_index()
    .rename(columns={"device_id": "unique_device_count"})
)

daily_device_counts.to_parquet("5.18_session_start_device_counts.parquet", index=False)

In [5]:
hackle_df4 = pd.read_parquet("/home/sprintda05/final/hackle/2023-05-24.parquet")

In [6]:
#  컬럼명 정리
hackle_df4 = hackle_df4.rename(columns={"Asia/Seoul": "datetime"})

#  $session_start만 필터링
session_start_df = hackle_df4[hackle_df4["event_key"] == "$session_start"].copy()

# datetime을 UTC로 간주 후 → KST로 변환
session_start_df["datetime"] = pd.to_datetime(session_start_df["datetime"], utc=True)
session_start_df["datetime_kst"] = session_start_df["datetime"].dt.tz_convert("Asia/Seoul")

# 날짜만 추출 (KST 기준)
session_start_df["date"] = session_start_df["datetime_kst"].dt.date

#  날짜별 유니크한 device_id 개수 집계
daily_device_counts = (
    session_start_df.groupby("date")["device_id"]
    .nunique()
    .reset_index()
    .rename(columns={"device_id": "unique_device_count"})
)

daily_device_counts.to_parquet("5.24_session_start_device_counts.parquet", index=False)

In [7]:
import json

In [8]:
file_path = "/home/sprintda05/final/hackle/2023-05-30.json"

with open(file_path, 'r', encoding='utf-8') as f:
    data = json.load(f)

In [9]:
hackle_df5 = pd.DataFrame(data)
hackle_df5

,Asia/Seoul,id,event_key,session_id,device_id,user_id,value,user_properties,hackle_properties,event_properties
0,2023-05-31T00:01:06.774Z,ea61928e-0642-4765-a5a7-6bf8f573784b,$session_start,VAb6ai91iKPZZQhGKenhHZpAG5B2,6DA6A8DC-1E61-4643-963A-ED3D782BE77A,None,0.0,{},"{'language': 'ko-KR', 'isapp': 'true', 'osvers...",{'sessionid': '1685458866774.78343b3e'}
1,2023-05-31T00:02:20.534Z,773f4cd7-00d1-4b79-bfe6-59788a145d01,$session_start,fe556647-de19-43ff-9152-b3c38a8b4d26,fe556647-de19-43ff-9152-b3c38a8b4d26,None,0.0,{},"{'language': 'ko', 'isapp': 'true', 'osversion...",{'sessionid': '1685458940534.47bbaa49'}
2,2023-05-31T00:02:46.447Z,8998b069-72fe-4a69-9b9c-310e7f631625,$session_start,LsAqqCTVtQPpevfSHXI8oRepVs73,5D47D549-EF22-41F3-A0D2-AD9EC1D21A2B,None,0.0,{},"{'language': 'ko-KR', 'isapp': 'true', 'osvers...",{'sessionid': '1685458966447.17499524'}
3,2023-05-31T00:03:50.029Z,bcc3c519-7de5-4fd4-a66c-ab348d73de61,$session_start,NBfi1jm5FKfB6We7YfVRHFgeJY83,FD419DE8-B3CE-4630-B234-959ADA5F47CF,None,0.0,{},"{'language': 'ko-KR', 'isapp': 'true', 'osvers...",{'sessionid': '1685459030029.9ef625ea'}
4,2023-05-31T00:04:12.317Z,14b3131e-3c30-4457-a52d-215441d8df1d,$session_start,3CBFC179-1BB1-40A2-84EF-C39EFBD40C14,3CBFC179-1BB1-40A2-84EF-C39EFBD40C14,None,0.0,{},"{'language': 'ko-KR', 'isapp': 'true', 'osvers...",{'sessionid': '1685459052317.fdf5aa30'}
...,...,...,...,...,...,...,...,...,...,...
2308,2023-06-05T23:28:56.499Z,fa3074ca-1502-4fde-bb5f-abe2df9aecdd,$session_start,39275C5A-CEF4-4182-807B-19B2495AB6BF,39275C5A-CEF4-4182-807B-19B2495AB6BF,None,0.0,{},"{'language': 'ko-KR', 'isapp': 'true', 'osvers...",{}
2309,2023-06-05T23:46:22.869Z,b8e03219-9769-40b1-82be-eb53a11af5cc,$session_start,9CCF2BAC-EAFA-45FD-AD19-61D656A4F0FF,9CCF2BAC-EAFA-45FD-AD19-61D656A4F0FF,None,0.0,{},"{'language': 'ko-KR', 'isapp': 'true', 'osvers...",{}
2310,2023-06-05T23:50:41.665Z,d1980f32-b974-4da9-8a90-fa4508ff6a5c,$session_start,Aq3CkkY1vRavVEaw8Wrftc3mqmL2,2C4D5EF8-A0FF-41FF-B816-05772A52163F,None,0.0,{},"{'language': 'ko-KR', 'isapp': 'true', 'osvers...",{}
2311,2023-06-05T23:53:28.191Z,3e2d6bda-52b9-49dc-b58a-22cae26cb43c,$session_start,UlkQSY7apmcmh6ZFhML1emdlZlC2,B9574AB1-D53E-4E3C-ADD5-202E90DE5D7D,None,0.0,{},"{'language': 'ko-KR', 'isapp': 'true', 'osvers...",{}


In [10]:
#  컬럼명 정리
hackle_df5 = hackle_df5.rename(columns={"Asia/Seoul": "datetime"})

#  $session_start만 필터링
session_start_df = hackle_df5[hackle_df5["event_key"] == "$session_start"].copy()

# datetime을 UTC로 간주 후 → KST로 변환
session_start_df["datetime"] = pd.to_datetime(session_start_df["datetime"], utc=True)
session_start_df["datetime_kst"] = session_start_df["datetime"].dt.tz_convert("Asia/Seoul")

# 날짜만 추출 (KST 기준)
session_start_df["date"] = session_start_df["datetime_kst"].dt.date

#  날짜별 유니크한 device_id 개수 집계
daily_device_counts = (
    session_start_df.groupby("date")["device_id"]
    .nunique()
    .reset_index()
    .rename(columns={"device_id": "unique_device_count"})
)

daily_device_counts.to_parquet("5.30_session_start_device_counts.parquet", index=False)

In [11]:
file_path1 = "/home/sprintda05/final/hackle/2023-06-05.json"

with open(file_path1, 'r', encoding='utf-8') as f:
    data = json.load(f)



In [12]:
hackle_df6 = pd.DataFrame(data)
hackle_df6

,Asia/Seoul,id,event_key,session_id,device_id,user_id,value,user_properties,hackle_properties,event_properties
0,2023-06-06T00:01:35.519Z,68f56e0b-4a74-4444-8c99-d5b4e9f41b2b,$session_start,7918DF48-A239-45CE-AC1E-7E2E356454C4,7918DF48-A239-45CE-AC1E-7E2E356454C4,None,0.0,{},"{'language': 'ko-KR', 'isapp': 'true', 'osvers...",{}
1,2023-06-06T00:16:11.790Z,e4572df7-3b39-445b-9653-19ff676cb66c,$session_start,02F51978-FDD9-4C66-AE8D-820943E00AEF,02F51978-FDD9-4C66-AE8D-820943E00AEF,None,0.0,{},"{'language': 'ko-KR', 'isapp': 'true', 'osvers...",{}
2,2023-06-06T00:27:07.180Z,fc00b675-848a-44d9-9a3c-81108214b774,$session_start,Ut9JqG0u4RQGihxUY3vjCTGgMIx2,E75FFCB7-5AAB-4674-AE34-7A231136C367,None,0.0,{},"{'language': 'ko-KR', 'isapp': 'true', 'osvers...",{}
3,2023-06-06T00:28:38.522Z,b95591f9-9291-40b5-91e6-83491f16566d,$session_start,92D752F2-E7B9-4C01-892B-C9B3F1416B74,92D752F2-E7B9-4C01-892B-C9B3F1416B74,None,0.0,{},"{'language': 'ko-KR', 'isapp': 'true', 'osvers...",{}
4,2023-06-06T00:44:06.441Z,d64634ec-34eb-4797-b7f8-db16ad1bab76,$session_start,47955d3d-b77f-47a6-9d3e-3fcb45a350df,47955d3d-b77f-47a6-9d3e-3fcb45a350df,None,0.0,{},"{'language': 'ko', 'isapp': 'true', 'osversion...",{}
...,...,...,...,...,...,...,...,...,...,...
644,2023-06-11T22:51:24.607Z,94ec9ec6-2631-4674-b3d4-658accbd0050,$session_start,dsSaWBfQEaTUSAj7dMa5I52pLJQ2,EEC79209-27BA-491B-AAFC-E31338E76BCE,None,0.0,{},"{'language': 'ko-KR', 'isapp': 'true', 'osvers...",{}
645,2023-06-11T22:57:30.920Z,bc377c37-3399-4639-a278-04146c6edec2,$session_start,tCowQfWVrPcXQmM1yhbl5qU6DyS2,30811F8A-CABB-4305-B62F-8E55B390AD61,None,0.0,{},"{'language': 'ko-KR', 'isapp': 'true', 'osvers...",{}
646,2023-06-11T22:59:16.152Z,8057403b-7d7a-47d4-9311-b812c6dc58b6,$session_start,E17ECC39-4D60-4751-8868-E8C69E82A485,E17ECC39-4D60-4751-8868-E8C69E82A485,None,0.0,{},"{'language': 'ko-KR', 'isapp': 'true', 'osvers...",{}
647,2023-06-11T23:16:34.479Z,7840bb35-f8f7-4c67-99a2-87931df445e5,$session_start,dbOAAiRxgCUQo2cKNFJ7HTNOGwB3,DAB953E5-76B2-4AC8-A69A-D66BFFD59A6A,None,0.0,{},"{'language': 'ko-KR', 'isapp': 'true', 'osvers...",{}


In [13]:
#  컬럼명 정리
hackle_df6 = hackle_df6.rename(columns={"Asia/Seoul": "datetime"})

#  $session_start만 필터링
session_start_df = hackle_df6[hackle_df6["event_key"] == "$session_start"].copy()

# datetime을 UTC로 간주 후 → KST로 변환
session_start_df["datetime"] = pd.to_datetime(session_start_df["datetime"], utc=True)
session_start_df["datetime_kst"] = session_start_df["datetime"].dt.tz_convert("Asia/Seoul")

# 날짜만 추출 (KST 기준)
session_start_df["date"] = session_start_df["datetime_kst"].dt.date

#  날짜별 유니크한 device_id 개수 집계
daily_device_counts = (
    session_start_df.groupby("date")["device_id"]
    .nunique()
    .reset_index()
    .rename(columns={"device_id": "unique_device_count"})
)

daily_device_counts.to_parquet("6.05_session_start_device_counts.parquet", index=False)

In [14]:
file_path2 = "/home/sprintda05/final/hackle/2023-06-11.json"

with open(file_path2, 'r', encoding='utf-8') as f:
    data = json.load(f)

In [15]:
hackle_df7 = pd.DataFrame(data)
hackle_df7

,Asia/Seoul,id,event_key,session_id,device_id,user_id,value,user_properties,hackle_properties,event_properties
0,2023-06-12T00:00:33.612Z,992ad7ef-4b37-4861-ab6e-845475eed154,$session_start,D25QsxpAUScwZKylPhMZ5GjRzWI3,8A5F41F9-D126-453C-8EA8-E0C2484584E3,None,0.0,{},"{'language': 'ko-KR', 'isapp': 'true', 'osvers...",{}
1,2023-06-12T00:10:18.093Z,8b4e52b5-213d-4c47-ad6b-b8cb6fcec039,$session_start,CiBKSEGugUW5yGj62SsXbFUzYTb2,A81A2A45-6260-41C6-B81B-071F006D60BD,None,0.0,{},"{'language': 'ko-KR', 'isapp': 'true', 'osvers...",{}
2,2023-06-12T00:17:21.980Z,52a8ba1d-d20a-4e55-b9e9-caae524bfc26,$session_start,CD41A81E-8D5C-4E7A-8289-587AD8C306F5,CD41A81E-8D5C-4E7A-8289-587AD8C306F5,None,0.0,{},"{'language': 'ko-KR', 'isapp': 'true', 'osvers...",{}
3,2023-06-12T00:21:00.694Z,eeff4d81-8313-4aad-bcce-85dc01e8aac2,$session_start,m55IE2zGHIVkRJv19Yzok0xivr03,5A92997F-362C-400B-8613-893A2FD6197F,None,0.0,{},"{'language': 'ko-KR', 'isapp': 'true', 'osvers...",{}
4,2023-06-12T00:22:44.843Z,e7929f29-7f73-41d2-af0a-7e1cff1ff863,$session_start,7FB46B64-B405-49C4-9378-8FABA73D4038,7FB46B64-B405-49C4-9378-8FABA73D4038,None,0.0,{},"{'language': 'ko-KR', 'isapp': 'true', 'osvers...",{}
...,...,...,...,...,...,...,...,...,...,...
258,2023-06-17T21:23:56.751Z,d9b8b8ff-3a3c-41c0-a001-de49603fe7e9,$session_start,sWYBfy29SMdfrhadzga4eEVYFbD3,90B5D332-DA86-41B2-9380-33B983F6F551,None,0.0,{},"{'language': 'ko-KR', 'isapp': 'true', 'osvers...",{}
259,2023-06-17T21:51:22.395Z,6866bc7f-eebe-4506-922e-e2c6d6a602d7,$session_start,8d771e46-b8f2-428a-a0aa-2b00201d8b03,8d771e46-b8f2-428a-a0aa-2b00201d8b03,None,0.0,{},"{'language': 'ko', 'isapp': 'true', 'osversion...",{}
260,2023-06-17T22:15:40.585Z,ab4a3ecb-6896-4722-b4a3-869a155ab80b,$session_start,dfbf1z46LQbVpYmdv4Rsb1P1nPt1,5CFC3146-E86E-4BD6-9A66-7C1E6AE66AB0,None,0.0,{},"{'language': 'ko-KR', 'isapp': 'true', 'osvers...",{}
261,2023-06-17T23:15:18.233Z,b1230ea8-03a9-4fb8-93cf-f8decece2366,$session_start,TZWdb8JcwmaMIqVlLDaAtH5x9lw2,07D3A812-6CEF-4449-B27F-7C47874877DD,None,0.0,{},"{'language': 'ko-KR', 'isapp': 'true', 'osvers...",{}


In [16]:
#  컬럼명 정리
hackle_df7 = hackle_df7.rename(columns={"Asia/Seoul": "datetime"})

#  $session_start만 필터링
session_start_df = hackle_df7[hackle_df7["event_key"] == "$session_start"].copy()

# datetime을 UTC로 간주 후 → KST로 변환
session_start_df["datetime"] = pd.to_datetime(session_start_df["datetime"], utc=True)
session_start_df["datetime_kst"] = session_start_df["datetime"].dt.tz_convert("Asia/Seoul")

# 날짜만 추출 (KST 기준)
session_start_df["date"] = session_start_df["datetime_kst"].dt.date

#  날짜별 유니크한 device_id 개수 집계
daily_device_counts = (
    session_start_df.groupby("date")["device_id"]
    .nunique()
    .reset_index()
    .rename(columns={"device_id": "unique_device_count"})
)

daily_device_counts.to_parquet("6.11_session_start_device_counts.parquet", index=False)

In [17]:
hackle_df8 = pd.read_parquet("/home/sprintda05/final/hackle/2023-06-17.parquet")

In [18]:
#  컬럼명 정리
hackle_df8 = hackle_df8.rename(columns={"Asia/Seoul": "datetime"})

#  $session_start만 필터링
session_start_df = hackle_df8[hackle_df8["event_key"] == "$session_start"].copy()

# datetime을 UTC로 간주 후 → KST로 변환
session_start_df["datetime"] = pd.to_datetime(session_start_df["datetime"], utc=True)
session_start_df["datetime_kst"] = session_start_df["datetime"].dt.tz_convert("Asia/Seoul")

# 날짜만 추출 (KST 기준)
session_start_df["date"] = session_start_df["datetime_kst"].dt.date

#  날짜별 유니크한 device_id 개수 집계
daily_device_counts = (
    session_start_df.groupby("date")["device_id"]
    .nunique()
    .reset_index()
    .rename(columns={"device_id": "unique_device_count"})
)

daily_device_counts.to_parquet("6.17_session_start_device_counts.parquet", index=False)

In [2]:
hackle_df9 = pd.read_parquet("/home/sprintda05/final/hackle/2023-06-23.parquet")

In [3]:
#  컬럼명 정리
hackle_df9 = hackle_df9.rename(columns={"Asia/Seoul": "datetime"})

#  $session_start만 필터링
session_start_df = hackle_df9[hackle_df9["event_key"] == "$session_start"].copy()

# datetime을 UTC로 간주 후 → KST로 변환
session_start_df["datetime"] = pd.to_datetime(session_start_df["datetime"], utc=True)
session_start_df["datetime_kst"] = session_start_df["datetime"].dt.tz_convert("Asia/Seoul")

# 날짜만 추출 (KST 기준)
session_start_df["date"] = session_start_df["datetime_kst"].dt.date

#  날짜별 유니크한 device_id 개수 집계
daily_device_counts = (
    session_start_df.groupby("date")["device_id"]
    .nunique()
    .reset_index()
    .rename(columns={"device_id": "unique_device_count"})
)

daily_device_counts.to_parquet("6.23_session_start_device_counts.parquet", index=False)

In [4]:
hackle_df10 = pd.read_parquet("/home/sprintda05/final/hackle/2023-06-29.parquet")

In [5]:
#  컬럼명 정리
hackle_df10 = hackle_df10.rename(columns={"Asia/Seoul": "datetime"})

#  $session_start만 필터링
session_start_df = hackle_df10[hackle_df10["event_key"] == "$session_start"].copy()

# datetime을 UTC로 간주 후 → KST로 변환
session_start_df["datetime"] = pd.to_datetime(session_start_df["datetime"], utc=True)
session_start_df["datetime_kst"] = session_start_df["datetime"].dt.tz_convert("Asia/Seoul")

# 날짜만 추출 (KST 기준)
session_start_df["date"] = session_start_df["datetime_kst"].dt.date

#  날짜별 유니크한 device_id 개수 집계
daily_device_counts = (
    session_start_df.groupby("date")["device_id"]
    .nunique()
    .reset_index()
    .rename(columns={"device_id": "unique_device_count"})
)

daily_device_counts.to_parquet("6.29_session_start_device_counts.parquet", index=False)

In [6]:
hackle_df11 = pd.read_parquet("/home/sprintda05/final/hackle/2023-07-05.parquet")

In [7]:
#  컬럼명 정리
hackle_df11 = hackle_df11.rename(columns={"Asia/Seoul": "datetime"})

#  $session_start만 필터링
session_start_df = hackle_df11[hackle_df11["event_key"] == "$session_start"].copy()

# datetime을 UTC로 간주 후 → KST로 변환
session_start_df["datetime"] = pd.to_datetime(session_start_df["datetime"], utc=True)
session_start_df["datetime_kst"] = session_start_df["datetime"].dt.tz_convert("Asia/Seoul")

# 날짜만 추출 (KST 기준)
session_start_df["date"] = session_start_df["datetime_kst"].dt.date

#  날짜별 유니크한 device_id 개수 집계
daily_device_counts = (
    session_start_df.groupby("date")["device_id"]
    .nunique()
    .reset_index()
    .rename(columns={"device_id": "unique_device_count"})
)

daily_device_counts.to_parquet("7.05_session_start_device_counts.parquet", index=False)